# Connecting to a Microsoft SQL Server with _pyodbc_

This is a **QUICK START** guide for the **pyobc** library for those who use it to connect to a **Microsoft SQL SERVER** database. However, this will not go into any detail, so you can and should read the full documentation [HERE](https://github.com/mkleehammer/pyodbc/wiki)

Besides the standard query stuff you would expect... there are A LOT of other things that you can do with pyodbc, with some creativity. For example, you can use it to stage an ETL process...yes, you can execute a python script with a scheduled task on your computer... pretty cool. I often run queries in a FOR loop when I want to process data over multiple periods of time, but it is more efficient to load it in one month or one year increments. The sky is the limit! Enjoy!

In [1]:
import pyodbc

## the connection string
Use the _pyodbc.drivers()_ method to find available drivers; the connection strings for these are essentially the same and will look like this:  

`conx = pyodbc.connect('DRIVER={SQL Server}; SERVER=TestServer; Database=TestDatabase; UID=UserID; PWD=Password;')`

If you use **Windows Authentication** to connect to the server, your string will look like this:  

`conx = pyodbc.connect('DRIVER={SQL Server}; SERVER=TestServer; Database=TestDatabase; TRUSTED_CONNECTION=yes')`

## setup the connection string & query variables

In [2]:
# find out what drivers you have available by using the `pyodbc.drivers()` method
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 13 for SQL Server', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server']


In [3]:
# create a variable that contains the connection string
conx_string = "driver={SQL SERVER}; server=GTLPF1MZF5M\IZZY_SQL_001; database=ADVENTUREWORKS2017; trusted_connection=YES;"

In [4]:
# create a variable that contains the sql query
query = "SELECT Name, CreditRating FROM Purchasing.Vendor WHERE CreditRating < 3"

## connect and extract data | individual steps

In [5]:
# create a connection object with the connection string
conx = pyodbc.connect(conx_string);

##### Create a cursor object that we can use to work in the database

In [6]:
cursor = conx.cursor();

##### Execute a query 

In [7]:
cursor.execute(query);

##### Store the query results in a variable

In [8]:
data = cursor.fetchall()

##### Display the first 5 rows on the query results

In [9]:
print(data[:5])

[('Australia Bike Retailer', 1), ('Allenson Cycles', 2), ('Advanced Bicycles', 1), ('Trikes, Inc.', 2), ('Morgan Bike Accessories', 1)]


##### Close the connection

In [10]:
conx.close()

## connect and extract data | consolidated with statement
##### Create a connection and import the data
There is no need to close the connection manually here as it will close when exiting the with statement

In [11]:
with pyodbc.connect(conx_string) as conx:
    cursor = conx.cursor()
    cursor.execute(query)
    data = cursor.fetchall()    
    
# display the first 5 records
print(data[:5])

[('Australia Bike Retailer', 1), ('Allenson Cycles', 2), ('Advanced Bicycles', 1), ('Trikes, Inc.', 2), ('Morgan Bike Accessories', 1)]


## other useful methods
##### Access the data as a named tuple, which can prove to be very handy

In [12]:
for row in data[:5]:
    print(f"Vendor Name   : {row.Name}\nCredit Rating : {row.CreditRating}")

Vendor Name   : Australia Bike Retailer
Credit Rating : 1
Vendor Name   : Allenson Cycles
Credit Rating : 2
Vendor Name   : Advanced Bicycles
Credit Rating : 1
Vendor Name   : Trikes, Inc.
Credit Rating : 2
Vendor Name   : Morgan Bike Accessories
Credit Rating : 1


##### Retrieve the column names for a table

In [13]:
# Retrieve the column names for a table
columns = [row.column_name for row in cursor.columns(table='Vendor')]
print(columns)

['BusinessEntityID', 'AccountNumber', 'Name', 'CreditRating', 'PreferredVendorStatus', 'ActiveFlag', 'PurchasingWebServiceURL', 'ModifiedDate']


##### Retrieve a list of tables in the database

In [14]:
tables = [row.table_name for row in cursor.tables()]
print(tables[:5]) # first 5 tables only for demonstration

['AWBuildVersion', 'DatabaseLog', 'ErrorLog', 'Department', 'Employee']


## select queries with parameters

##### **SINGLE** parameter query

In [15]:
# select female employees
with pyodbc.connect(conx_string) as conx:
    cursor = conx.cursor()
    cursor.execute('SELECT NationalIDNumber, JobTitle FROM HumanResources.Employee WHERE Gender = ?', 'F')
    f_emp_data = cursor.fetchall()
    
# display the first 5 records
for row in f_emp_data[:5]:
    print(row)

('245797967', 'Vice President of Engineering')
('695256908', 'Design Engineer')
('811994146', 'Research and Development Engineer')
('658797903', 'Research and Development Engineer')
('486228782', 'Tool Designer')


##### **MULTI** parameters query

In [16]:
# select male employees who have fewer than 40 vacation hours
with pyodbc.connect(conx_string) as conx:
    cursor = conx.cursor()
    cursor.execute('SELECT NationalIDNumber, JobTitle, VacationHours FROM HumanResources.Employee WHERE Gender = ? AND VacationHours < ?' , ('M',40))
    m_vac_data = cursor.fetchall()
    
# display the first 5 records
for row in m_vac_data[:5]:
    print(row)

('509647174', 'Engineering Manager', 2)
('998320692', 'Design Engineer', 6)
('879342154', 'Research and Development Manager', 16)
('974026903', 'Senior Tool Designer', 7)
('480168528', 'Tool Designer', 9)


## insert queries

In [17]:
# create a variable that contains the connection string
conx_string_b = "driver={SQL SERVER}; server=GTLPF1MZF5M\IZZY_SQL_001; database=SANDBOX; trusted_connection=YES;"

##### **SINGLE** record insert

In [18]:
with pyodbc.connect(conx_string_b) as conx:
    cursor = conx.cursor()
    cursor.execute("INSERT INTO VacationExceptions (NationalIDNumber, JobTitle, VacationHours) VALUES(125467985467854, 'Head Hancho', 500)")  

##### _Verify the load with a select query_

In [19]:
with pyodbc.connect(conx_string_b) as conx:
    cursor = conx.cursor()
    cursor.execute("SELECT * FROM VacationExceptions")
    data = cursor.fetchall()

print(data)

[('125467985467854', 'Head Hancho', 500)]


##### **MULTI** record insert FROM VALUES

In [20]:
exceptions = [('615389812','Sales Representative',150),('982310417','European Sales Manager',75)]

with pyodbc.connect(conx_string_b) as conx:
    cursor = conx.cursor()
    cursor.executemany("INSERT INTO VacationExceptions (NationalIDNumber, JobTitle, VacationHours) VALUES(?,?,?)", exceptions)

##### _Verify the load with a select query (1 record from single insert, and 2 records for multi insert... 3 total)_

In [21]:
with pyodbc.connect(conx_string_b) as conx:
    cursor = conx.cursor()
    cursor.execute("SELECT * FROM VacationExceptions")
    data = cursor.fetchall()

for row in data:
    print(row)

('125467985467854', 'Head Hancho', 500)
('615389812', 'Sales Representative', 150)
('982310417', 'European Sales Manager', 75)


##### **MULTI** record insert from ANOTHER QUERY result

In [22]:
# load the male vacation exceptions dataset into the VacationExceptions table
with pyodbc.connect(conx_string_b) as conx:
    cursor = conx.cursor()
    cursor.executemany("INSERT INTO VacationExceptions (NationalIDNumber, JobTitle, VacationHours) VALUES(?,?,?)", m_vac_data)

##### _Verify the load with a select query (first 10 records)_

In [23]:
with pyodbc.connect(conx_string_b) as conx:
    cursor = conx.cursor()
    cursor.execute("SELECT * FROM VacationExceptions")
    data = cursor.fetchall()

for row in data[:10]:
    print(row)

('125467985467854', 'Head Hancho', 500)
('615389812', 'Sales Representative', 150)
('982310417', 'European Sales Manager', 75)
('509647174', 'Engineering Manager', 2)
('998320692', 'Design Engineer', 6)
('879342154', 'Research and Development Manager', 16)
('974026903', 'Senior Tool Designer', 7)
('480168528', 'Tool Designer', 9)
('42487730', 'Senior Design Engineer', 3)
('14417807', 'Production Technician - WC60', 21)
